In [2]:
from re import I
import torch
import torch.nn as nn
from torch.nn import functional as F

In [17]:
eval_interval = 500
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 10
max_iters = 5000
n_head = 6
n_layer = 2
dropout = 0.2
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

In [ ]:
# shakespear dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
from embeddings import MyEncoder

my_encoder = MyEncoder(text)
data = torch.tensor(my_encoder.tokens, dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([301829]) torch.int64
tensor([ 3488,  7161,   349,  5685,   408,  5738,   678,  3131,     5,  4145,
          552,  4028,   233,  1797,   349, 11796,     5,  4028,   233,  3488,
         7161,   349,  1922,   360,   493,  8942,  3201,   164,  2015,   844,
          164,  1604,   604,  1501,  1797,   349,   723,     7,  8942,   233,
         3488,  7161,   349,  3488,     5,   339,  1095,   207,  1639,   206,
         2099,  3747,   206,   225,  5666,  5242,   164,   136,   970,   233,
         1797,   349,  1285,  1095,   722,     5,   408,  1095,   722,   233,
         3488,  7161,   349,  5618,   423,  3568,  1110,     5,   175,   408,
         2351,   434,  7008,   354,   798,  1407,  2398,   443,  2695,   722,
          121,  2590,  1501,  1797,   349,  2019,   596,  4459,   240,   722,
           12,   832,   278,   238,  2063,    11,  2255,     5,  2255,  1687,
         7671,  7161,   349,  2763,  2437,     5,  1292,  5710,   233,  3488,
         7161,   349,  1285,   

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
torch.manual_seed(1337)
# batch_size = 4 # how many independent sequences will we process in parallel?
# block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y



In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
from embeddings import NGramLanguageModeler

ngmodel = NGramLanguageModeler(my_encoder.vocab_size, embedding_dim=n_embd, context_size=2)
ngmodel.load_state_dict(torch.load("embedding_model.pt"))
ngmodel.embeddings

Embedding(12111, 10)

In [19]:
from gpte import BigramLanguageModel
torch.manual_seed(1337)

model = BigramLanguageModel(my_encoder.vocab_size, embeddings=ngmodel.embeddings)
m = model.to(device)



In [ ]:

logits, loss = m(xb, yb)
print(logits.shape)
print(logits)
print(loss)

In [20]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [24]:
for step in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if step % eval_interval == 0 or step == max_iters - 1:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

torch.Size([8, 384]) torch.Size([32, 8, 10])


RuntimeError: The size of tensor a (10) must match the size of tensor b (384) at non-singleton dimension 2

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))